In [3]:
import heapq

def solve_maze(maze):
    # Encontra as posições de início e fim
    start = None
    end = None
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 2:
                start = (i, j)
            elif maze[i][j] == 3:
                end = (i, j)
    
    if not start or not end:
        return None
    
    # Direções possíveis (cima, baixo, esquerda, direita)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    # Fila de prioridade para o algoritmo A*
    open_set = []
    heapq.heappush(open_set, (0, start))
    
    # Dicionários para armazenar o caminho e custos
    came_from = {}
    g_score = {start: 0}  # Custo real do caminho do início até o nó
    f_score = {start: heuristic(start, end)}  # Custo estimado total (g + h)
    
    while open_set:
        current = heapq.heappop(open_set)[1]
        
        # Chegamos ao destino
        if current == end:
            return reconstruct_path(came_from, current)
        
        # Explora vizinhos
        for di, dj in directions:
            neighbor = (current[0] + di, current[1] + dj)
            
            # Verifica se o vizinho é válido
            if not is_valid(maze, neighbor):
                continue
            
            # Custo temporário do caminho
            tentative_g_score = g_score[current] + 1
            
            # Se encontramos um caminho melhor para este vizinho
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, end)
                
                # Adiciona à fila de prioridade se não estiver lá
                if neighbor not in [item[1] for item in open_set]:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    # Não encontrou caminho
    return None

def is_valid(maze, pos):
    """Verifica se uma posição é válida no labirinto."""
    rows, cols = len(maze), len(maze[0])
    i, j = pos
    
    # Verifica limites do labirinto
    if i < 0 or i >= rows or j < 0 or j >= cols:
        return False
    
    # Verifica se não é parede ou minotauro
    return maze[i][j] != 1 and maze[i][j] != 9

def heuristic(pos, end):
    """Função heurística (distância de Manhattan) para o A*."""
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def reconstruct_path(came_from, current):
    """Reconstrói o caminho do fim até o início."""
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

def print_maze_with_path(maze, path):
    """Imprime o labirinto com o caminho marcado."""
    maze_copy = [row[:] for row in maze]
    
    # Marca o caminho (exceto início e fim)
    for i, j in path[1:-1]:
        maze_copy[i][j] = 4  # Usamos 4 para representar o caminho
    
    for row in maze_copy:
        print(' '.join(str(cell) for cell in row))

In [2]:
# Labirinto de exemplo (mesmo do enunciado)
maze = [
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 2, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 9, 1],
    [1, 0, 1, 3, 1, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1]
]

# Função para imprimir o labirinto de forma visual
def print_maze(m):
    for row in m:
        print(" ".join(str(cell) for cell in row))
    print()

print("Labirinto original:")
print_maze(maze)

# Executa o solver
path = solve_maze(maze)

if path:
    print(f"\nCaminho encontrado com {len(path)-1} movimentos:")
    
    # Cria uma cópia do labirinto para marcar o caminho
    maze_with_path = [row[:] for row in maze]
    for step, (i, j) in enumerate(path):
        if maze[i][j] == 0:  # Só marca se não for início, fim ou obstáculo
            maze_with_path[i][j] = 4  # Usa 4 para representar o caminho
    
    print_maze(maze_with_path)
    
    print("Coordenadas do caminho:")
    for i, j in path:
        print(f"({i}, {j})", end=" → ")
    print("SAÍDA")
else:
    print("Não foi possível encontrar um caminho para a saída.")

Labirinto original:
1 1 1 1 1 1 1 1
1 2 0 0 0 0 0 1
1 1 1 1 1 0 1 1
1 0 0 0 0 0 0 1
1 0 1 1 1 1 0 1
1 0 0 0 0 0 9 1
1 0 1 3 1 0 1 1
1 1 1 1 1 1 1 1


Caminho encontrado com 15 movimentos:
1 1 1 1 1 1 1 1
1 2 4 4 4 4 0 1
1 1 1 1 1 4 1 1
1 4 4 4 4 4 0 1
1 4 1 1 1 1 0 1
1 4 4 4 0 0 9 1
1 0 1 3 1 0 1 1
1 1 1 1 1 1 1 1

Coordenadas do caminho:
(1, 1) → (1, 2) → (1, 3) → (1, 4) → (1, 5) → (2, 5) → (3, 5) → (3, 4) → (3, 3) → (3, 2) → (3, 1) → (4, 1) → (5, 1) → (5, 2) → (5, 3) → (6, 3) → SAÍDA


In [10]:
import cvxpy as cp
import numpy as np

def print_maze_with_path(maze, path):
    """Imprime o labirinto com o caminho marcado."""
    maze_copy = [row[:] for row in maze]
    
    # Marca o caminho (exceto início e fim)
    for i, j in path[1:-1]:
        maze_copy[i][j] = 4  # Usamos 4 para representar o caminho
    
    for row in maze_copy:
        print(' '.join(str(cell) for cell in row))

def solve_maze(maze):
    """Resolve o problema do labirinto usando PLI."""
    m, n = maze.shape
    
    # Encontrar posições inicial (2) e final (3)
    pos_inicial = tuple(np.argwhere(maze == 2)[0])
    pos_final = tuple(np.argwhere(maze == 3)[0])
    
    # Número máximo de passos (limitado pelo tamanho do labirinto)
    T = m * n
    
    # Variável de decisão: x[i,j,t] = 1 se o agente está na posição (i,j) no tempo t
    x = cp.Variable((m, n, T), boolean=True)
    
    # Função objetivo: minimizar o tempo de chegada ao destino
    objetivo = cp.Minimize(sum(t * x[pos_final[0], pos_final[1], t] for t in range(T)))
    
    # Restrições
    restricoes = []
    
    # 1. Começa na posição inicial no tempo 0
    restricoes.append(x[pos_inicial[0], pos_inicial[1], 0] == 1)
    
    # 2. Não pode estar em outras posições no tempo 0
    for i in range(m):
        for j in range(n):
            if (i,j) != pos_inicial:
                restricoes.append(x[i,j,0] == 0)
    
    # 3. Chega exatamente uma vez na posição final
    restricoes.append(sum(x[pos_final[0], pos_final[1], t] for t in range(T)) == 1)
    
    # 4. Não pode estar em paredes (1) ou no Minotauro (9) em nenhum tempo
    for i in range(m):
        for j in range(n):
            if maze[i,j] in [1, 9]:
                for t in range(T):
                    restricoes.append(x[i,j,t] == 0)
    
    # 5. Movimentação entre células adjacentes
    for t in range(1, T):
        for i in range(m):
            for j in range(n):
                if maze[i,j] in [0, 2, 3]:  # Apenas células livres, início ou fim
                    # O agente só pode estar aqui se veio de uma célula vizinha no passo anterior
                    vizinhos = []
                    for di, dj in [(-1,0), (1,0), (0,-1), (0,1)]:  # Movimentos: cima, baixo, esquerda, direita
                        ni, nj = i + di, j + dj
                        if 0 <= ni < m and 0 <= nj < n:
                            vizinhos.append(x[ni,nj,t-1])
                    
                    if len(vizinhos) > 0:
                        restricoes.append(x[i,j,t] <= sum(vizinhos))
    
    # 6. O agente não pode desaparecer (se chegou ao destino, permanece lá)
    for t in range(1, T):
        restricoes.append(x[pos_final[0], pos_final[1], t] >= x[pos_final[0], pos_final[1], t-1])
    
    # 7. O agente deve estar em exatamente uma posição em cada tempo
    for t in range(T):
        restricoes.append(sum(x[i,j,t] for i in range(m) for j in range(n)) == 1)
    
    # Resolver o problema
    prob = cp.Problem(objetivo, restricoes)
    prob.solve(solver=cp.SCIP)  # Pode usar também CPLEX ou ECOS_BB
    
    # Extrair o caminho da solução
    path = []
    if prob.status == cp.OPTIMAL:
        # Encontrar a ordem cronológica das posições
        for t in range(T):
            for i in range(m):
                for j in range(n):
                    if x[i,j,t].value > 0.5:  # Considera como 1 se > 0.5
                        path.append((i, j))
                        break
        
        # Remover posições repetidas consecutivas (quando fica parado)
        unique_path = []
        for pos in path:
            if not unique_path or pos != unique_path[-1]:
                unique_path.append(pos)
        
        return unique_path
    else:
        return None

# Definir o labirinto conforme seu exemplo esperado
labirinto = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 2, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 9, 1],
    [1, 0, 1, 3, 1, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1]
])

print("Labirinto original:")
print_maze_with_path(labirinto, [])

solution_path = solve_maze(labirinto)

if solution_path:
    print("\nLabirinto com caminho solução:")
    print_maze_with_path(labirinto, solution_path)
    print("\nPassos do caminho:", solution_path)
else:
    print("Não foi encontrada solução para o labirinto.")

Labirinto original:
1 1 1 1 1 1 1 1
1 2 0 0 0 0 0 1
1 1 1 1 1 0 1 1
1 0 0 0 0 0 0 1
1 0 1 1 1 1 0 1
1 0 0 0 0 0 9 1
1 0 1 3 1 0 1 1
1 1 1 1 1 1 1 1


/home/pedro.trindade/anaconda3/envs/optmization/lib/python3.13/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(



Labirinto com caminho solução:
1 1 1 1 1 1 1 1
1 2 4 4 4 4 0 1
1 1 1 1 1 4 1 1
1 4 4 4 4 4 0 1
1 4 1 1 1 1 0 1
1 4 4 4 0 0 9 1
1 0 1 3 1 0 1 1
1 1 1 1 1 1 1 1

Passos do caminho: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (3, 4), (3, 3), (3, 2), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (5, 1), (5, 2), (5, 3), (6, 3)]
